# *Thermal Transport in a Ti:Sapphire Crystal*

In this demo, we solve the three-dimensional diffusion equation in a short
cylindrical Ti:sapphire crystal, with Dirichlet boundary conditions $u_D$
and steady source term $f$ along the central axis.

$$
\begin{align}
  u'   &= \alpha\nabla^2 u        \quad\text{in a short cylinder} \\
  u    &= u_D  \hphantom{Du}\quad\text{on the boundary} \\
  u    &= u_0  \hphantom{Du}\quad\;\text{at $t = 0$}
\end{align}
$$
with
$$
\begin{align}
  u_D  &= 0 \\
  u_0  &= f \\
  f    &= A \exp\bigl( -(r/\rho)^2 - (z/\zeta)^2 \bigr).
\end{align}
$$

## Typical Properties of sapphire wafers

[data source](http://valleydesign.com/sappprop.htm)


### Mechanical Properties
| property                     | value                        |
| :--------------------------- | :--------------------------- |
| Density                      | 3.98 g/cm$^3$                |
| Young's Modulus (elasticity) | 345 GPa / $50\times10^6$ psi |
| Bulk Modulus (compression)   | 250 GPa / $36\times10^6$ psi |
| Shear Modulus (rigidity)     | 145 GPa / $21\times10^6$ psi |
| Poisson's Ratio              | 0.29                         |
| Melting Point                | 2040°C / 3700°F / 2310 K     |
 
### Thermal Properties
| property                   | value                 |
| :------------------------- | :-------------------- |
| Thermal Conductivity       |                       |
| $\quad$ Perpendicular to C | at 23°C: 23.0  W/m°C  |
|                            | at 77°C: 16.8  W/m°C  |
| $\quad$ Parallel to C      | at 24°C: 25.8  W/m°C  |
|                            | at 70°C: 17.35 W/m°C  |
| Specific Heat              | at 18°C: 756 J/kg°C   |
|                            | at -182°C: 104 J/kg°C |

### Electrical Properties
| property                   | value                    |
| :------------------------- | :----------------------- |
| Volume Resistivity         | at 25°C $10^{14}$ ohm/cm |
| Dielectric Strength        | $4.8 \times 10^5$        |
| Dielectric Constant:       |                          |
| $\quad$ Perpendicular to C | 9.4                      |
| $\quad$ Parallel to C      | 11.5                     |
| Dissipation Factor         | $10^-4\tan\delta$        |


### CTE (Coefficient of Thermal Expansion), $\ \times 10^6 /\,{}^\circ\text{C}$
| Temp °C | Perpendicular to C | Parallel to C |
| ------: | -----------------: | ------------: |
|      70 |               6.95 |          5.90 |
|     100 |               7.08 |          6.05 |
|     200 |               7.66 |          6.60 |
|     300 |               8.30 |          7.32 |
|     400 |               9.00 |          8.07 |
|     500 |               9.63 |          8.88 |
|     600 |              10.45 |          9.77 |

#### diffusion constant
Make sure we have the units right! They ought to be area/time:
$$
  \alpha = \frac{\kappa}{\rho c_p} \approx
           \frac{17~\text{W}\,/\,(\text{m} ˚\text{C})}{
                 3.98\times10^3~\text{kg}\,/\,\text{m}^3 \cdot 756~\text{J}\,/\,(\text{kg} ˚\text{C})}
         \approx 5.65\times10^{-6}~\text{m}^2\,/\,\text{s}
         \approx 5.65\times10^{-2}~\text{cm}^2\,/\,\text{s}
$$

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt
import plotly.graph_objects
import numpy as np
# import math as m

from fenics import *
from mshr import *   # now deprecated (?), transition to alternate mesh generator (?)

In [ ]:
def minmax(it):
    min = max = None
    for val in it:
        if min is None or val < min:
            min = val
        if max is None or val > max:
            max = val
    return min, max

In [18]:
## set up the problem and define (some of) the main parameters

# simulation parameters
# -- crystal properties
a_saph = 5.65e-2  # cm^2/s diffusion constant of sapphire
a_perp = 5.58e-2  # perpendicular to C
a_para = 5.77e-2  # parallel to C
# -- time step
T = 5.0           # total simulation time / sec
n_steps = 100     # number of time steps
dt = T / n_steps  # size of time step
nip = 10          # number of intervals between records
# -- crystal dimensions
diam = 7.0        # diameter
leng = 2.0        # length
# --  mesh density
md = 35           # mesh density within cylinder

# derived parameters
rad = diam / 2  # radius
lh  = leng / 2  # half-length

# scaling
# -- omega & k (wave-number)
omega = 1.                       # rad / s
wvnum = np.sqrt(omega / a_saph)  # 1 / cm
# -- scaled dimensions
rad_scl  = wvnum * diam / 2      # scaled radius
lh_scl   = wvnum * leng / 2      # scaled half-length
# -- scaled time
T_scl  = omega * T   # scaled simulation time
dt_scl = omega * dt  # scaled time-step

print("duration:  ", T)
print("time-step: ", dt)
print("plot-step: ", dt * nip)

duration:   5.0
time-step:  0.05
plot-step:  0.5


In [ ]:
# create mesh: Cylinder(p_top, p_bot, r_top, r_bot)
# ### not yet ### # -- do this in scaled coördinates
# domain = Cylinder(Point(0, 0, lh_scl), Point(0., 0., -lh_scl), rad_scl, rad_scl)
domain = Cylinder(Point(0, 0, lh), Point(0., 0., -lh), rad, rad)
mesh = generate_mesh(domain, md)
# and define function space
V = FunctionSpace(mesh, 'P', 1)

In [ ]:
mesh

In [ ]:
# define boundary condition
def boundary(x, on_boundary):
    return on_boundary

bc = DirichletBC(V, Constant(0.), boundary)

# define initial value
A = 72.0         # amplitude
rho =  rad / 10  # radial decay length
zeta = lh / 4    # longitudinal decay length
u_0 = Expression('A * exp( -(pow(x[0],2) + pow(x[1],2)) / pow(rho,2) - pow(x[2]/zeta,2) )',
                 degree=1, A=A, rho=rho, zeta=zeta)
u_n = interpolate(u_0, V)

# define the variational problem: u' = D.∆u
u = TrialFunction(V)
v = TestFunction(V)
f = Constant(0)
F = u*v*dx + dt*a_saph*dot(grad(u), grad(v))*dx - (u_n + dt*f)*v*dx
a, L = lhs(F), rhs(F)

In [ ]:
# define time-evolution function
def evolve():

    # report initial state
    yield u_n

    # time-stepping
    u = Function(V)
    t = 0
    for n in range(1, n_steps + 1):

        # update current time
        t += dt

        # compute solution
        solve(a == L, u, bc)

        # report solution at nip-step intervals
        if n % nip == 0:
            yield u

        # update previous solution
        u_n.assign(u)

In [ ]:
# get the "facets" and build an array of the indices of their coordinates
inds = []
for item in dolfin.cpp.mesh.facets(mesh):
    inds.append(item.entities(0).tolist())

# we will provide these indices to plotly so it can draw proper surfaces
inds = np.array(inds)
ii = inds[:, 0]
jj = inds[:, 1]
kk = inds[:, 2]

# get node coordinate values
xvals = mesh.coordinates()[:,0]
yvals = mesh.coordinates()[:,1]
zvals = mesh.coordinates()[:,2]
# and ranges
xmin, xmax = xvals.min(), xvals.max()
ymin, ymax = yvals.min(), yvals.max()
zmin, zmax = zvals.min(), zvals.max()

In [ ]:
n_rows = 1
n_cols = 2
# fig_wd = 15
# default sizing here yields unit aspect ratio
# plt.figure(figsize = (fig_wd, fig_wd * n_rows // n_cols))
# plt.subplot(n_rows, n_cols, idx)

plt.figure(figsize=(13,8))

tol = 3.e-3  # avoid hitting points outside the domain
xv = np.linspace(xmin * (1 - tol), xmax * (1 - tol), 51)
zv = np.linspace(zmin * (1 - tol), zmax * (1 - tol), 21)
radpts = [(x_, 0, 0) for x_ in xv]
axipts = [(0, 0, z_) for z_ in zv]

idx = 0
for u in evolve():
    idx += 1
    ux = np.array([u(pt) for pt in radpts])
    uz = np.array([u(pt) for pt in axipts])
    plt.subplot(n_rows, n_cols, 1)
    plt.plot(xv, ux, lw=1)
    plt.subplot(n_rows, n_cols, 2)
    plt.plot(zv, uz, lw=1)  

plt.subplot(n_rows, n_cols, 1)
plt.plot(xv, ux, 'k', lw=1)
plt.subplot(n_rows, n_cols, 2)
plt.plot(zv, uz, 'k', lw=1)  
plt.show()

In [ ]:
plt.figure(figsize=(9.6, 6))

plt.subplot(2, 2, 1)
plt.plot(xvals, yvals
         mx_trk[0::4, mx_col['X']][1:-1] * 100, c='tab:blue', lw=0.5)
plt.plot(mx_trk[0::4, mx_col['S']][1:-1],
         mx_trk[0::4, mx_col['X']][1:-1] * 100, '.', label=r'$mx:Y^\prime$')
# plt.ylabel(r'$X_h\,/\,\mathrm{cm}$')
# plt.xlabel(r'$s\,/\,\mathrm{m}$')
# plt.legend()
# plt.grid()
# plt.xlim(xrng)
# plt.ylim(yrng)
# plt.axvline(ud3_e, c='g', lw=0.5)
# plt.axvline(ud3_s, c='r', lw=0.5)

...

plt.subplot(2, 2, 4)
plt.plot(mx_trk[0::4, mx_col['S'] ][1:-1],
         mx_trk[0::4, mx_col['PY']][1:-1] * 1000, c='tab:blue', lw=0.5)
plt.plot(fai_data[:, fai_col['S'] ] / 100,
         fai_data[:, fai_col['P'] ], c='orange', lw=0.5)
plt.plot(mx_trk[0::4, mx_col['S'] ][1:-1],
         mx_trk[0::4, mx_col['PY']][1:-1] * 1000, '.', label=r'$mx:Y^\prime$')
plt.plot(fai_data[:, fai_col['S'] ] / 100,
         fai_data[:, fai_col['P'] ], '.', label=r'$zg:Z^\prime$')
plt.ylabel(r'$X_v^\prime\,/\,\mathrm{mr}$')
plt.xlabel(r'$s\,/\,\mathrm{m}$')
plt.legend()
plt.grid()
plt.xlim(xrng)
plt.ylim(yprng)
plt.axvline(ud3_e, c='g', lw=0.5)
plt.axvline(ud3_s, c='r', lw=0.5)

plt.tight_layout()
plt.show()

In [ ]:
uvals = u.compute_vertex_values()

fig = plotly.graph_objects.Figure(
    data=[plotly.graph_objects.Mesh3d(
        x=xvals, y=yvals, z=zvals,
        i=ii, j=jj, k=kk,
        colorscale='jet',
        intensity=uvals,
        alphahull=-1,
        opacity=0.02
    )]
)
fig.update_layout(scene_aspectmode='data')
fig.show()